## **Objective:** 

This is the MLB Exploratory Data Analytics notebook of [Alok Patni](https://www.kaggle.com/alokpattani/mlb-player-digital-engagement-data-exploration) , and my main objective is to understand the data extraction, feature creation and importance of individual features (factors) on target variables.

## **Competition Goal:**
The MLB data is in csv and json format, main goal is to predict the target variables (1, 2, 3, 4), so called (digital engagement) for next day from player, events, games, teams and twitter follower dataset.

## **Notebook Flow**
1. Handle multi row, column data and transform them in pandas dataframe from csv and json format.
2. Data preprocessing (data transformation, data cleaning and pre analytics)
3. Use feature engineering to generate new features, dataframes which leads to primary result
4. Finding factor (feature) importance matrix with respect to targets
5. Derive conclusion from result


# Import Libraries and Data from Competition API

In [1]:
# import required libraries
import os
import sys
import gc
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
        
import plotly as pl
import plotly.express as plx
import plotly.offline as ploff
import plotly.graph_objs as plgo 

import ipywidgets as widgets
from pathlib import Path

# Expand max column width when displaying data frames and want all the columns of dataset
pd.set_option('display.max_colwidth', 100)
pd.set_option("display.max_columns", None)          

#### **Read Kaggle Files**: 

Training Dataset is in json format and each cell contains value of player, target, games and awards aswell, first we need to extract data from json format and convert into dataframe for further analysis. So, first we make the function for unpacking the data from jsion to pandas dataframe.

In [2]:
# creating data set (training) 
def unpack_json(json_str):
  return pd.DataFrame() if pd.isna(json_str) else pd.read_json(json_str)

# unpack json data
def unpack_data(data, dfs=None, n_jobs=-1):
  if dfs is not None:
    data = data.loc[:,dfs]
  unnested_dfs = {}
  for name, column in data.iteritems():
    daily_dfs = Parallel(n_jobs = n_jobs)(delayed(unpack_json)(item) for date, item in column.iteritems())
    df = pd.concat(daily_dfs)
    unnested_dfs[name] = df
  return unnested_dfs

'''Why we use delayed, while performing the parallel (joblib) operation? : TO delay the internal operation on thread, so parallel job can get the data to perform.
---> https://stackoverflow.com/questions/42220458/what-does-the-delayed-function-do-when-used-with-joblib-in-python'''

'Why we use delayed, while performing the parallel (joblib) operation? : TO delay the internal operation on thread, so parallel job can get the data to perform.\n---> https://stackoverflow.com/questions/42220458/what-does-the-delayed-function-do-when-used-with-joblib-in-python'

In [3]:
# Import Data Path...
input_file_path = Path('../input/mlb-player-digital-engagement-forecasting')

csv_and_df_names = pd.DataFrame(data= {'csv_name' : ['seasons', 'teams', 'players', 'awards', 'example_test', 'example_sample_submission'], 
                                       'df_name': ['seasons', 'teams', 'players', 'award_pre2018', 'example_test', 'example_sample_submission']})

# set up tabbed output
data_tabs = widgets.Tab()

# Add output widgets fo each EF as tabs' children
data_tabs.children = list([widgets.Output() for df_name in csv_and_df_names['df_name']])

for index, row in csv_and_df_names.iterrows():
    csv_name = row['csv_name']
    df_name = row['df_name']

  # Read from csv and create df with specified name in environment
    globals()[df_name] = pd.read_csv(input_file_path/f"{csv_name}.csv")
    data_tabs.set_title(index, df_name)
    
    with data_tabs.children[index]:
        display(eval(df_name))

print('Competition Data Files')
display(data_tabs)

Competition Data Files


#### **Process:** 
Change date format of train data from (20180101)  to standard (YMD- 2018-01-01), help to understand easily...

In [4]:
train = pd.read_csv('../input/mlb-player-digital-engagement-forecasting/train.csv')
train['date'] = pd.to_datetime(train['date'], format = "%Y%m%d")
display(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1216 entries, 0 to 1215
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   date                     1216 non-null   datetime64[ns]
 1   nextDayPlayerEngagement  1216 non-null   object        
 2   games                    639 non-null    object        
 3   rosters                  1216 non-null   object        
 4   playerBoxScores          538 non-null    object        
 5   teamBoxScores            538 non-null    object        
 6   transactions             1103 non-null   object        
 7   standings                531 non-null    object        
 8   awards                   294 non-null    object        
 9   events                   536 non-null    object        
 10  playerTwitterFollowers   40 non-null     object        
 11  teamTwitterFollowers     40 non-null     object        
dtypes: datetime64[ns](1), object(11)
m

None

### Extract Data from train.csv

In [6]:
# Get names of all "nested" data frames in daily training set
daily_data_nested_df_names = train.drop('date', axis = 1).columns.values.tolist()
# display(daily_data_nested_df_names)                                                        # main column name in train dataset...


for df_name in daily_data_nested_df_names:
    date_nested_table = train[['date', df_name]]
    date_nested_table = date_nested_table[~pd.isna(date_nested_table[df_name])].reset_index(drop = True)                 # ~ sign meaning: only return value except NAN...
    
    daily_dfs_collection = []
    
    for date_index, date_row in date_nested_table.iterrows():
        daily_df = pd.read_json(date_row[df_name])                                 # start basic dataset(daily_df) and extract date from each df_name column...
        daily_df['dailyDataDate'] = date_row['date']     
        daily_dfs_collection = daily_dfs_collection + [daily_df]

    # Concatenate all daily dfs into single df for each row
    unnested_table = pd.concat(daily_dfs_collection, ignore_index = True).set_index('dailyDataDate').reset_index()       # Set and reset index to move 'dailyDataDate' to front of df
    
    
    # Creates 1 pandas df per unnested df from daily data read in, with same name
    globals()[df_name] = unnested_table    
    
    # Clean up tables and collection of daily data frames for this df (RAM free) 
    del(date_nested_table, daily_dfs_collection, unnested_table)

    
# ==================================<< Presentaed Output >> =======================================
# Set up for tabbed output
daily_data_unnested_tabs = widgets.Tab()

# Add Output widgets for each (eventual) DF as tabs' children
daily_data_unnested_tabs.children = list([widgets.Output() 
  for df_name in daily_data_nested_df_names])

# display extracted data one by one as widget...
for index in range(0, len(daily_data_nested_df_names)):
    df_name = daily_data_nested_df_names[index]
    # Rename tab bar titles to df names
    daily_data_unnested_tabs.set_title(index, df_name)
   # Display corresponding table output for this tab name
    with daily_data_unnested_tabs.children[index]:
        display(eval(df_name))

display(daily_data_unnested_tabs)

In [7]:
# activate garbage collector.
del train              # free some RAM
gc.collect() 

22

### Exploratory Analysis of Target Variables

The main motto of this competition is to predict the target variables (in scale of 0 to 100) for fiding the digital engagement. So, let's understand the target variable distribution from the train dataset.

In [8]:
# Melt data to get " 1 row per player date per target , inshort melt data to get per target row (seperate targets) 
player_engagement_targets_melted = pd.melt(nextDayPlayerEngagement,
                                           id_vars = ['dailyDataDate','playerId'],
                                           value_vars = ['target1', 'target2', 'target3', 'target4'],
                                           var_name = 'target')
# display(player_engagement_targets_melted)

# calculate some distribution summary value by target
player_engagement_data_summary_by_target = (player_engagement_targets_melted
                                            .groupby(['target'], as_index = False)
                                            .agg(count = ('value', 'count'), 
                                                 mean = ('value', np.mean),
                                                 stdDev= ('value', np.std),
                                                 pctle10 = ('value', lambda x: np.percentile(x, q = 10)),
                                                 pctle25 = ('value', lambda x: np.percentile(x, q = 25)),
                                                 median =  ('value', np.median),
                                                 pctle75 = ('value', lambda x: np.percentile(x, q = 75)),
                                                 pctle90 = ('value', lambda x: np.percentile(x, q = 90)),
                                                 pctValues0 = ('value', lambda  x: np.mean(x==0 )* 100),
                                                 pctValues100=('value', lambda x: np.mean(x==100)* 100)
                                                 ))

display(player_engagement_data_summary_by_target.round(decimals = 3))

,target,count,mean,stdDev,pctle10,pctle25,median,pctle75,pctle90,pctValues0,pctValues100
0,target1,2506176,0.569,4.172,0.000,0.000,0.001,0.018,0.379,37.806,0.049
1,target2,2506176,2.456,6.235,0.005,0.077,0.556,2.236,6.042,8.329,0.049
2,target3,2506176,0.688,5.065,0.000,0.000,0.002,0.021,0.253,37.972,0.049
3,target4,2506176,1.137,4.230,0.000,0.054,0.223,0.758,2.297,10.598,0.049


### Visualize the target based distribution to understand the pattern of "player digital engagement with targets"

In [11]:
# convert the target values to rounded value...
player_engagement_targets_melted['roundedValue'] = (player_engagement_targets_melted['value'].round(0))

# group data via target and rounded value so we can get " number of players by roundedvalue per target" ===> Ex: target1's roundedvalue=0 has n number of players
player_engagement_targets_dist = (player_engagement_targets_melted.groupby(['target', 'roundedValue'], as_index = False).agg(numPlayerDates = ('playerId', 'count')))

# add another column ('cumPlayerDates') via adding all the numPlayerDates togather ... 
player_engagement_targets_dist['cumPlayerDates'] = (player_engagement_targets_dist.groupby(['target'])['numPlayerDates'].cumsum()) 

# devide by total Number of player (getting ratio of cumplayerdates to total players)
player_engagement_targets_dist['cumPctPlayerDates'] = (player_engagement_targets_dist['cumPlayerDates'] / nextDayPlayerEngagement.shape[0]) * 100 # total distance / total_data

# visualize data for simple understanding and interpretation...
player_engagement_targets_dist_plot = plx.bar(player_engagement_targets_dist, 
                                             x = 'roundedValue',
                                             y = 'numPlayerDates',
                                             facet_row = 'target',
                                             hover_data = player_engagement_targets_dist.columns,
                                              labels = {
                                                 'roundedValue'     : 'Rounded Target Value',
                                                 'numPlayerDates'   : '# of players',
                                                 'cumPlayerDates'   : 'cumulative # of Player-Dates',
                                                 'cumPctPlayerDates':'Cumulative % of Player-Dates'
                                             },
                                             title = 'Target Value Distributions across player-dates',
                                             width = 900,
                                             height= 900
                                             )

ploff.iplot(player_engagement_targets_dist_plot)


### Look at correlations among target metrics across player-dates

In [12]:
player_engagement_targets_correlations = (nextDayPlayerEngagement[['target1', 'target2', 'target3', 'target4']].corr())

display(player_engagement_targets_correlations.round(decimals = 3))

,target1,target2,target3,target4
target1,1.000,0.405,0.411,0.385
target2,0.405,1.000,0.388,0.549
target3,0.411,0.388,1.000,0.370
target4,0.385,0.549,0.370,1.000


In [13]:
# remove large melted player engagement data frame to clear some memory
del(player_engagement_targets_melted)
gc.collect()

3584

### Exploratory Analysis and preparation of dat for potential predictors of digital engagement

#### Dates Relative to Season

In [14]:
# Extract some information on daily base from daily data ( using the season date of interest)
# date dataframe
dates = pd.DataFrame(data = {'dailyDataDate' : nextDayPlayerEngagement['dailyDataDate'].unique()})


# seperate year and month from main date format for analysis
dates['year'] = dates['dailyDataDate'].dt.year
dates['month'] = dates['dailyDataDate'].dt.month

# create a cumulative data frame for further analytics on (data to season exploration) 
dates_with_info = pd.merge(dates, seasons, left_on = 'year', right_on = 'seasonId')     # seasonId is Years (joining component)

# count anything between regular and postseason as "in season" ----> finding date between (regular season start date < x < post season end date)
dates_with_info['inSeason'] = (dates_with_info['dailyDataDate'].between(dates_with_info['regularSeasonStartDate'], dates_with_info['postSeasonEndDate'],inclusive = True))

# determine season type based on following logic .. [ex: if dailydatadate (individual row date) < pre season start date then we will call it as offseason record.]
dates_with_info['seasonPart'] = np.select(
                                          [
                                            dates_with_info['dailyDataDate'] < dates_with_info['preSeasonStartDate'], 
                                            dates_with_info['dailyDataDate'] < dates_with_info['regularSeasonStartDate'],
                                            dates_with_info['dailyDataDate'] <= dates_with_info['lastDate1stHalf'],
                                            dates_with_info['dailyDataDate'] < dates_with_info['firstDate2ndHalf'], 
                                            dates_with_info['dailyDataDate'] <= dates_with_info['regularSeasonEndDate'],
                                            dates_with_info['dailyDataDate'] < dates_with_info['postSeasonStartDate'],
                                            dates_with_info['dailyDataDate'] <= dates_with_info['postSeasonEndDate'],
                                            dates_with_info['dailyDataDate'] > dates_with_info['postSeasonEndDate']
  ],
  [
    'Offseason',
    'Preseason',
    'Reg Season 1st Half',
    'All-Star Break',
    'Reg Season 2nd Half',
    'Between Reg and Postseason',
    'Postseason',
    'Offseason'
  ], 
  default = np.nan
  )

display(dates_with_info.head(), dates_with_info.shape)

# select required column only...
dates_with_season_part = (dates_with_info[['dailyDataDate', 'year','seasonId', 'month', 'inSeason', 'seasonPart']]
                          .rename(columns = {'seasonId': 'season'}))         # seasonID -->> season

display(dates_with_season_part.head(), dates_with_season_part.shape)

,dailyDataDate,year,month,seasonId,seasonStartDate,seasonEndDate,preSeasonStartDate,preSeasonEndDate,regularSeasonStartDate,regularSeasonEndDate,lastDate1stHalf,allStarDate,firstDate2ndHalf,postSeasonStartDate,postSeasonEndDate,inSeason,seasonPart
0,2018-01-01,2018,1,2018,2018-03-29,2018-10-28,2018-02-21,2018-03-27,2018-03-29,2018-10-01,2018-07-15,2018-07-17,2018-07-19,2018-10-02,2018-10-28,False,Offseason
1,2018-01-02,2018,1,2018,2018-03-29,2018-10-28,2018-02-21,2018-03-27,2018-03-29,2018-10-01,2018-07-15,2018-07-17,2018-07-19,2018-10-02,2018-10-28,False,Offseason
2,2018-01-03,2018,1,2018,2018-03-29,2018-10-28,2018-02-21,2018-03-27,2018-03-29,2018-10-01,2018-07-15,2018-07-17,2018-07-19,2018-10-02,2018-10-28,False,Offseason
3,2018-01-04,2018,1,2018,2018-03-29,2018-10-28,2018-02-21,2018-03-27,2018-03-29,2018-10-01,2018-07-15,2018-07-17,2018-07-19,2018-10-02,2018-10-28,False,Offseason
4,2018-01-05,2018,1,2018,2018-03-29,2018-10-28,2018-02-21,2018-03-27,2018-03-29,2018-10-01,2018-07-15,2018-07-17,2018-07-19,2018-10-02,2018-10-28,False,Offseason


(1216, 17)

,dailyDataDate,year,season,month,inSeason,seasonPart
0,2018-01-01,2018,2018,1,False,Offseason
1,2018-01-02,2018,2018,1,False,Offseason
2,2018-01-03,2018,2018,1,False,Offseason
3,2018-01-04,2018,2018,1,False,Offseason
4,2018-01-05,2018,2018,1,False,Offseason


(1216, 6)

## Analyze the Roster dataset

* This data were extracted from the train dataset (via json extraction)
* 'rosters' is the list of player registered for teams who will play the season. 

In [15]:
# understand data
display(rosters.head())

,dailyDataDate,playerId,gameDate,teamId,statusCode,status
0,2018-01-01,400121,2018-01-01,116,A,Active
1,2018-01-01,408045,2018-01-01,142,A,Active
2,2018-01-01,425492,2018-01-01,120,A,Active
3,2018-01-01,429664,2018-01-01,136,A,Active
4,2018-01-01,431151,2018-01-01,121,A,Active


In [16]:
# check the unique roster status code ...
print(rosters['status'].unique())

# distribute the nubmer of the players according to status (unique status = 16) ...
roster_status_values = (rosters.groupby(['statusCode', 'status'], as_index = False).agg(numPlayerDates = ('playerId', 'count')))
roster_status_values = roster_status_values.sort_values(['numPlayerDates'],ascending = False, ignore_index = True)
roster_status_values['checkdisSum'] = roster_status_values['numPlayerDates'].cumsum()  # just to check the total number of records are correct (not missing), [checkdisSum = rosters' # sum]
display(roster_status_values, roster_status_values.shape)

['Active' 'Injured 60-Day' 'Reassigned to Minors' 'Injured 10-Day'
 'Injured 7-Day' 'Paternity List' 'Suspended # days'
 'Family Medical Emergency' 'Bereavement List' 'Reserve List (Minors)'
 'Deceased' '60-day IL' '10-day IL' 'Reassigned' 'Paternity' 'Suspended']


,statusCode,status,numPlayerDates,checkdisSum
0,A,Active,1185608,1185608
1,RM,Reassigned to Minors,191572,1377180
2,D60,Injured 60-Day,55063,1432243
3,D10,Injured 10-Day,45037,1477280
4,RM,Reassigned,7413,1484693
5,D10,10-day IL,2242,1486935
6,D60,60-day IL,1498,1488433
7,D7,Injured 7-Day,475,1488908
8,PL,Paternity List,258,1489166
9,SU,Suspended # days,247,1489413


(16, 4)

### Check the duplicacy in roster rows per player-data

In [17]:
# group the players on daily date, and count playerId according to date. If playerId > 1 means, there is duplicacy in roster data.
player_dates_multiple_roster_entries = rosters.groupby(['dailyDataDate', 'playerId'], as_index = False).agg(numPlayerDateRosterEntries = ('playerId', 'count'))
player_dates_multiple_roster_entries = player_dates_multiple_roster_entries.query("numPlayerDateRosterEntries > 1")
  
display(player_dates_multiple_roster_entries)

,dailyDataDate,playerId,numPlayerDateRosterEntries


Conclusion: The rows are zero, means there is no duplicacy in the dataset.

## Analyze Team Game and Game Stats
* This data were extracted from the train dataset (via json extraction)
* has detailed reocrd of indivudual game entry...

In [18]:
# visualize data...
display(games.head())
gamesStateCodes = games['codedGameState'].unique()
gameTypes = games['gameType'].unique()
display(gamesStateCodes)
display(gameTypes)

,dailyDataDate,gamePk,gameType,season,gameDate,gameTimeUTC,resumeDate,resumedFrom,codedGameState,detailedGameState,isTie,gameNumber,doubleHeader,dayNight,scheduledInnings,gamesInSeries,seriesDescription,homeId,homeName,homeAbbrev,homeWins,homeLosses,homeWinPct,homeWinner,homeScore,awayId,awayName,awayAbbrev,awayWins,awayLosses,awayWinPct,awayWinner,awayScore
0,2018-02-21,533782,E,2018,2018-02-21,2018-02-21T20:10:00Z,NaN,NaN,F,Final,0.0,1,N,day,7,0.0,Exhibition,109,Arizona Diamondbacks,ARI,1,0,1.0,True,7.0,5035,Arizona State Sun Devils,ASU,0.0,1.0,0.0,False,2.0
1,2018-02-22,534461,E,2018,2018-02-22,2018-02-22T18:05:00Z,NaN,NaN,F,Final,0.0,1,N,day,9,0.0,Exhibition,116,Detroit Tigers,DET,1,0,1.0,True,6.0,228,Florida Southern College Mocs,FSC,0.0,1.0,0.0,False,1.0
2,2018-02-22,545334,E,2018,2018-02-22,2018-02-22T18:05:00Z,NaN,NaN,F,Final,0.0,1,N,day,9,0.0,Exhibition,143,Philadelphia Phillies,PHI,1,0,1.0,True,6.0,231,University of Tampa Spartans,UT,0.0,1.0,0.0,False,0.0
3,2018-02-22,547295,E,2018,2018-02-22,2018-02-22T03:33:00Z,NaN,NaN,F,Final,0.0,2,Y,night,9,0.0,Exhibition,111,Boston Red Sox,BOS,2,0,1.0,True,4.0,227,Boston College Eagles,BC,0.0,1.0,0.0,False,2.0
4,2018-02-22,533784,E,2018,2018-02-22,2018-02-22T23:05:00Z,NaN,NaN,F,Final,0.0,1,N,night,9,0.0,Exhibition,142,Minnesota Twins,MIN,1,0,1.0,True,2.0,4864,Minnesota Gophers,UM,0.0,1.0,0.0,False,1.0


array(['F', 'C', 'D', 'S', 'U', 'O'], dtype=object)

array(['E', 'S', 'R', 'A', 'F', 'D', 'L', 'W'], dtype=object)

#### Turn games table into 1 row per team game, then merge the with team box score so we can get team insights with performance...

In [19]:
# Filter all  regular and all-star game marked "final" in game table...
games_for_stats = games[np.isin(games['gameType'], ['R', 'F', 'D', 'L', 'W', 'C', 'P', 'A']) & (games['codedGameState'] == 'F')]
# display(games_for_stats.head(), games_for_stats.shape)

# 1. Home team
games_home_perspective = games_for_stats.copy()
games_home_perspective.columns = [col_value.replace('home','team').replace('away', 'opp') for col_value in games_home_perspective.columns.values]
games_home_perspective['isHomeTeam'] = 1

# 2. Away Team
games_away_perspective = games_for_stats.copy()
games_away_perspective.columns = [col_value.replace('home','opp').replace('away','team') for col_value in games_away_perspective.columns.values]
games_away_perspective['isHomeTeam'] = 0

# concating both the dataframes (row wise operation)
team_games = (pd.concat([games_home_perspective, games_away_perspective], ignore_index = True))
# display(team_games.head(), team_games.shape)

# teamBox Score merging...
team_game_stats = teamBoxScores.copy()
team_game_stats.columns = [(col_value + 'Team') if (col_value not in ['dailyDataDate', 'home', 'teamId', 'gamePk', 'gameDate', 'gameTimeUTC']) 
                           else col_value for col_value in team_game_stats.columns.values]
# display(team_game_stats.head(), team_game_stats.shape)

# merge logically generated data with extracted game dataset
team_games_with_stats = pd.merge(team_games, team_game_stats.drop(['home','gameTimeUTC'], axis =1), 
                                 on = ['dailyDataDate', 'gamePk', 'gameDate', 'teamId'], how = 'inner') # inner joint on list of columns by droping some of columns.

display(team_games_with_stats.head(), team_games_with_stats.shape)

,dailyDataDate,gamePk,gameType,season,gameDate,gameTimeUTC,resumeDate,resumedFrom,codedGameState,detailedGameState,isTie,gameNumber,doubleHeader,dayNight,scheduledInnings,gamesInSeries,seriesDescription,teamId,teamName,teamAbbrev,teamWins,teamLosses,teamWinPct,teamWinner,teamScore,oppId,oppName,oppAbbrev,oppWins,oppLosses,oppWinPct,oppWinner,oppScore,isHomeTeam,flyOutsTeam,groundOutsTeam,runsScoredTeam,doublesTeam,triplesTeam,homeRunsTeam,strikeOutsTeam,baseOnBallsTeam,intentionalWalksTeam,hitsTeam,hitByPitchTeam,atBatsTeam,caughtStealingTeam,stolenBasesTeam,groundIntoDoublePlayTeam,groundIntoTriplePlayTeam,plateAppearancesTeam,totalBasesTeam,rbiTeam,leftOnBaseTeam,sacBuntsTeam,sacFliesTeam,catchersInterferenceTeam,pickoffsTeam,airOutsPitchingTeam,groundOutsPitchingTeam,runsPitchingTeam,doublesPitchingTeam,triplesPitchingTeam,homeRunsPitchingTeam,strikeOutsPitchingTeam,baseOnBallsPitchingTeam,intentionalWalksPitchingTeam,hitsPitchingTeam,hitByPitchPitchingTeam,atBatsPitchingTeam,caughtStealingPitchingTeam,stolenBasesPitchingTeam,inningsPitchedTeam,earnedRunsTeam,battersFacedTeam,outsPitchingTeam,hitBatsmenTeam,balksTeam,wildPitchesTeam,pickoffsPitchingTeam,rbiPitchingTeam,inheritedRunnersTeam,inheritedRunnersScoredTeam,catchersInterferencePitchingTeam,sacBuntsPitchingTeam,sacFliesPitchingTeam
0,2018-03-29,529408,R,2018,2018-03-29,2018-03-29T20:15:00Z,NaN,NaN,F,Final,0.0,1,N,day,9,2.0,Regular Season,118,Kansas City Royals,KC,0.0,1.0,0.0,False,7.0,145,Chicago White Sox,CWS,1.0,0.0,1.0,True,14.0,1,6,13,7,3,0,1,2,2,0,9,1,36,0,0,0,0,39,15,6,9,0,0,0,0,0,0,14,0,0,6,7,7,0,14,1,39,0,0,9.0,14,47,27,1,0,0,0,14,0,0,0,0,0
1,2018-03-29,529410,R,2018,2018-03-29,2018-03-30T02:10:00Z,NaN,NaN,F,Final,0.0,1,N,night,9,3.0,Regular Season,109,Arizona Diamondbacks,ARI,1.0,0.0,1.0,True,8.0,115,Colorado Rockies,COL,0.0,1.0,0.0,False,2.0,1,4,9,8,2,1,0,11,6,0,12,0,36,0,2,0,0,42,16,8,28,0,0,0,0,0,0,2,0,0,2,12,2,0,9,0,33,0,0,9.0,2,36,27,0,0,0,0,2,0,0,0,1,0
2,2018-03-29,529411,R,2018,2018-03-29,2018-03-29T19:35:00Z,NaN,NaN,F,Final,0.0,1,N,day,9,4.0,Regular Season,140,Texas Rangers,TEX,0.0,1.0,0.0,False,1.0,117,Houston Astros,HOU,1.0,0.0,1.0,True,4.0,1,9,4,1,1,0,0,9,2,0,6,1,31,0,0,0,0,34,7,0,12,0,0,0,1,0,0,4,0,0,2,13,6,0,6,0,30,0,0,9.0,4,37,27,0,0,0,0,4,0,0,0,0,1
3,2018-03-29,529418,R,2018,2018-03-29,2018-03-29T23:08:00Z,NaN,NaN,F,Final,0.0,1,N,day,9,4.0,Regular Season,119,Los Angeles Dodgers,LAD,0.0,1.0,0.0,False,0.0,137,San Francisco Giants,SF,1.0,0.0,1.0,True,1.0,1,3,11,0,0,0,0,10,5,0,6,0,31,0,2,0,0,36,6,0,16,0,0,0,0,0,0,1,0,0,1,11,2,0,8,0,34,0,0,9.0,1,36,27,0,0,0,0,1,0,0,0,0,0
4,2018-03-29,529407,R,2018,2018-03-29,2018-03-29T16:40:00Z,NaN,NaN,F,Final,0.0,1,N,day,9,4.0,Regular Season,146,Miami Marlins,MIA,0.0,1.0,0.0,False,4.0,112,Chicago Cubs,CHC,1.0,0.0,1.0,True,8.0,1,4,13,4,2,1,0,7,6,0,8,1,33,0,0,0,0,40,12,4,21,0,0,0,0,0,0,8,0,0,3,10,5,0,9,3,35,0,0,9.0,7,46,27,3,0,0,0,8,0,0,1,0,0


(12528, 86)

#### **Process:**

* Aggregate team game-level stats to daily date stat (... check the duplicacy about the per day game entry)
* Note: If the output has 0 rows, then we can procceed without worrying the duplicate data. 

In [24]:
# verify that no team played 2 different opponenets or 2 different game types allows opp and game type to be used in agregation w/0 getting multiple towsx
team_date_gameTypes_opps_agg = team_games_with_stats.groupby(['dailyDataDate', 'teamId'], as_index = False).agg(numGameTypes = ('gameType', 'nunique'), numOppIds = ('oppId', 'nunique'), numOppNames = ('oppName', 'nunique'))      # unique can consider 0, nunique can not...

# Can proceed without worrying about duplicate team-dates as long as this returns 0 rows
display(team_date_gameTypes_opps_agg[
    (team_date_gameTypes_opps_agg['numGameTypes'] != 1) |(team_date_gameTypes_opps_agg['numOppIds'] != 1) | (team_date_gameTypes_opps_agg['numOppNames'] != 1)
  ])
print('If the above is zero means, there is no duplicacy')

,dailyDataDate,teamId,numGameTypes,numOppIds,numOppNames


If the above is zero means, there is no duplicacy


In [25]:
# 
team_date_stats_agg = (team_games_with_stats.groupby(['dailyDataDate', 'teamId', 'gameType', 'oppId', 'oppName'], as_index = False)
                       .agg(numGamesTeam = ('gamePk', 'nunique'), winsTeam = ('teamWinner', 'sum'), lossesTeam = ('oppWinner', 'sum'), runsScoredTeam = ('teamScore', 'sum'), runsAllowedTeam = ('oppScore', 'sum')))

display(team_date_stats_agg.head(), team_date_stats_agg.shape)

,dailyDataDate,teamId,gameType,oppId,oppName,numGamesTeam,winsTeam,lossesTeam,runsScoredTeam,runsAllowedTeam
0,2018-03-29,108,R,133,Oakland Athletics,1,0,1,5.0,6.0
1,2018-03-29,109,R,115,Colorado Rockies,1,1,0,8.0,2.0
2,2018-03-29,110,R,142,Minnesota Twins,1,1,0,3.0,2.0
3,2018-03-29,111,R,139,Tampa Bay Rays,1,0,1,4.0,6.0
4,2018-03-29,112,R,146,Miami Marlins,1,1,0,8.0,4.0


(12258, 10)


Add some information to player game level stats...

## Player Game Stat: 
* add some stat/ info to player game-level stats 

In [26]:
# copy player box score and rename the field and rename the columns...
player_game_stats = playerBoxScores.copy().rename(columns = {'teamId': 'gameTeamId', 'teamName':'gameTeamName'})
# display(player_game_stats.head(), player_game_stats.shape)

# Adds in field for innings pitched as fraction (better for aggregation) ---> avoid blank value(NAN) and modify other data as per the logic formula...
player_game_stats['inningsPitchedAsFrac'] = np.where(pd.isna(player_game_stats['inningsPitched']),
                                                     np.nan,
                                                     np.floor(player_game_stats['inningsPitched']) + (player_game_stats['inningsPitched'] - np.floor(player_game_stats['inningsPitched'])) * 10/3)
display(player_game_stats.head(), player_game_stats.shape)

# Add in Tom Tango pitching game score (https://www.mlb.com/glossary/advanced-stats/game-score) ---- > np.where(condition, ifTrue->X, ifFalse->Y)
player_game_stats['pitchingGameScore'] = np.where(
  # pitching game score doesn't apply if player didn't pitch, set to NA and calculate the score via SME info...
    pd.isna(player_game_stats['pitchesThrown']) | (player_game_stats['pitchesThrown'] == 0),
    np.nan,
    (40 + 2 * player_game_stats['outsPitching'] + 1 * player_game_stats['strikeOutsPitching'] - 2 * player_game_stats['baseOnBallsPitching'] - 2 * player_game_stats['hitsPitching'] - 3 * player_game_stats['runsPitching'] - 6 * player_game_stats['homeRunsPitching']))

# Look at top pitching game scores in span of data
player_game_top_pitching_game_scores = player_game_stats[['gameDate', 'playerName', 'gameTeamName', 'outsPitching','strikeOutsPitching', 'baseOnBallsPitching', 'hitsPitching','runsPitching', 'homeRunsPitching', 'pitchingGameScore']].sort_values(['pitchingGameScore'], ascending = False,ignore_index = True)

print('Top Pitching Game Scores in span of Data')
display(player_game_top_pitching_game_scores.head(10))
print('')

# add in criteria for no-hitter by pitcher (individual, not multiple pitchers)
player_game_stats['noHitter'] = np.where((player_game_stats['completeGamesPitching'] == 1) & (player_game_stats['inningsPitched'] >= 9) & (player_game_stats['hitsPitching'] == 0), 1, 0)

# generate no-hitter dataset with requried column for future analytcis...
player_game_no_hitters = (player_game_stats[player_game_stats['noHitter'] == 1][['gameDate','playerName','gameTeamName', 'completeGamesPitching', 'inningsPitched', 'hitsPitching', 'noHitter','pitchingGameScore']]
                          .sort_values(['gameDate'], ascending = False, ignore_index = True))

print('Individual No-Hitters in span of Data')
display(player_game_no_hitters.head(10), player_game_no_hitters.shape)

,dailyDataDate,home,gamePk,gameDate,gameTimeUTC,gameTeamId,gameTeamName,playerId,playerName,jerseyNum,positionCode,positionName,positionType,battingOrder,gamesPlayedBatting,flyOuts,groundOuts,runsScored,doubles,triples,homeRuns,strikeOuts,baseOnBalls,intentionalWalks,hits,hitByPitch,atBats,caughtStealing,stolenBases,groundIntoDoublePlay,groundIntoTriplePlay,plateAppearances,totalBases,rbi,leftOnBase,sacBunts,sacFlies,catchersInterference,pickoffs,gamesPlayedPitching,gamesStartedPitching,completeGamesPitching,shutoutsPitching,winsPitching,lossesPitching,flyOutsPitching,airOutsPitching,groundOutsPitching,runsPitching,doublesPitching,triplesPitching,homeRunsPitching,strikeOutsPitching,baseOnBallsPitching,intentionalWalksPitching,hitsPitching,hitByPitchPitching,atBatsPitching,caughtStealingPitching,stolenBasesPitching,inningsPitched,saveOpportunities,earnedRuns,battersFaced,outsPitching,pitchesThrown,balls,strikes,hitBatsmen,balks,wildPitches,pickoffsPitching,rbiPitching,gamesFinishedPitching,inheritedRunners,inheritedRunnersScored,catchersInterferencePitching,sacBuntsPitching,sacFliesPitching,saves,holds,blownSaves,assists,putOuts,errors,chances,inningsPitchedAsFrac
0,2018-03-29,1,529418,2018-03-29,2018-03-29T23:08:00Z,119,Los Angeles Dodgers,605131,Austin Barnes,15,12,Pinch Runner,Runner,601.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-03-29,1,529406,2018-03-29,2018-03-29T20:00:00Z,139,Tampa Bay Rays,605480,Mallex Smith,0,7,Outfielder,Outfielder,601.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN
2,2018-03-29,0,529416,2018-03-29,2018-03-29T20:10:00Z,143,Philadelphia Phillies,546318,Odubel Herrera,37,8,Outfielder,Outfielder,401.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN
3,2018-03-29,0,529412,2018-03-29,2018-03-29T20:05:00Z,108,Los Angeles Angels,527043,Jefry Marte,19,3,First Base,Infielder,402.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,NaN
4,2018-03-29,1,529408,2018-03-29,2018-03-29T20:15:00Z,118,Kansas City Royals,449181,Paulo Orlando,16,8,Outfielder,Outfielder,701.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,0.0,2.0,NaN


(185144, 87)

Top Pitching Game Scores in span of Data


,gameDate,playerName,gameTeamName,outsPitching,strikeOutsPitching,baseOnBallsPitching,hitsPitching,runsPitching,homeRunsPitching,pitchingGameScore
0,2019-09-01,Justin Verlander,Houston Astros,27.0,14.0,1.0,0.0,0.0,0.0,106.0
1,2018-05-04,Gerrit Cole,Houston Astros,27.0,16.0,1.0,1.0,0.0,0.0,106.0
2,2021-04-23,Jacob deGrom,New York Mets,27.0,15.0,0.0,2.0,0.0,0.0,105.0
3,2020-08-25,Lucas Giolito,Chicago White Sox,27.0,13.0,1.0,0.0,0.0,0.0,105.0
4,2021-04-09,Joe Musgrove,San Diego Padres,27.0,10.0,0.0,0.0,0.0,0.0,104.0
5,2019-04-14,German Marquez,Colorado Rockies,27.0,9.0,0.0,1.0,0.0,0.0,101.0
6,2021-04-14,Carlos Rodon,Chicago White Sox,27.0,7.0,0.0,0.0,0.0,0.0,101.0
7,2019-06-05,Chris Sale,Boston Red Sox,27.0,12.0,0.0,3.0,0.0,0.0,100.0
8,2018-04-09,Max Scherzer,Washington Nationals,27.0,10.0,0.0,2.0,0.0,0.0,100.0
9,2019-07-24,Shane Bieber,Cleveland Indians,27.0,10.0,1.0,1.0,0.0,0.0,100.0



Individual No-Hitters in span of Data


,gameDate,playerName,gameTeamName,completeGamesPitching,inningsPitched,hitsPitching,noHitter,pitchingGameScore
0,2021-04-14,Carlos Rodon,Chicago White Sox,1.0,9.0,0.0,1,101.0
1,2021-04-09,Joe Musgrove,San Diego Padres,1.0,9.0,0.0,1,104.0
2,2020-09-13,Alec Mills,Chicago Cubs,1.0,9.0,0.0,1,93.0
3,2020-08-25,Lucas Giolito,Chicago White Sox,1.0,9.0,0.0,1,105.0
4,2019-09-01,Justin Verlander,Houston Astros,1.0,9.0,0.0,1,106.0
5,2019-05-07,Mike Fiers,Oakland Athletics,1.0,9.0,0.0,1,96.0
6,2018-05-08,James Paxton,Seattle Mariners,1.0,9.0,0.0,1,95.0
7,2018-04-21,Sean Manaea,Oakland Athletics,1.0,9.0,0.0,1,100.0


(8, 8)

Aggregate player game-evel stats to daily date(accounts for multiple games per day)

In [27]:
check = player_game_stats['gameTeamName'].min()
print(check)

American League All-Stars


In [28]:
# mergeing two datasets
player_date_stats_agg = pd.merge((player_game_stats.groupby(['dailyDataDate','playerId'], as_index=False)
                                .agg(numGames=('gamePk','nunique'),
                                    numTeams = ('gameTeamId', 'nunique'),
                                    gameTeamId = ('gameTeamId', 'min'),
                                    gameTeamName = ('gameTeamName', 'min'))), 
                                 
                                (player_game_stats.groupby(['dailyDataDate', 'playerId'], as_index= False)[[  
                                'gamesPlayedBatting', 'runsScored', 'doubles', 'triples', 'homeRuns','strikeOuts', 'baseOnBalls', 'hits', 'hitByPitch', 
                                    'atBats','caughtStealing', 'stolenBases', 'groundIntoDoublePlay','groundIntoTriplePlay', 'plateAppearances', 'totalBases',
                                    'rbi','gamesPlayedPitching', 'gamesStartedPitching', 'completeGamesPitching','shutoutsPitching', 'winsPitching', 'lossesPitching', 
                                    'runsPitching','homeRunsPitching', 'strikeOutsPitching', 'baseOnBallsPitching','hitsPitching', 'earnedRuns', 'battersFaced', 'outsPitching',
                                    'pitchesThrown', 'balls', 'strikes', 'saves', 'holds', 'blownSaves','inningsPitchedAsFrac', 'pitchingGameScore', 'noHitter','assists', 
                                    'putOuts', 'errors' ]].sum())
                                 ,on = ['dailyDataDate', 'playerId']
                                 ,how= 'inner')

display(player_date_stats_agg.head(),player_date_stats_agg.shape)

,dailyDataDate,playerId,numGames,numTeams,gameTeamId,gameTeamName,gamesPlayedBatting,runsScored,doubles,triples,homeRuns,strikeOuts,baseOnBalls,hits,hitByPitch,atBats,caughtStealing,stolenBases,groundIntoDoublePlay,groundIntoTriplePlay,plateAppearances,totalBases,rbi,gamesPlayedPitching,gamesStartedPitching,completeGamesPitching,shutoutsPitching,winsPitching,lossesPitching,runsPitching,homeRunsPitching,strikeOutsPitching,baseOnBallsPitching,hitsPitching,earnedRuns,battersFaced,outsPitching,pitchesThrown,balls,strikes,saves,holds,blownSaves,inningsPitchedAsFrac,pitchingGameScore,noHitter,assists,putOuts,errors
0,2018-03-29,134181,1,1,140,Texas Rangers,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,1.0,1.0,0.0
1,2018-03-29,400085,1,1,136,Seattle Mariners,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,0.0,2.0,0.0
2,2018-03-29,400284,1,1,119,Los Angeles Dodgers,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,0.0,0.0,0.0
3,2018-03-29,405395,1,1,108,Los Angeles Angels,1.0,1.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,2.0,6.0,0.0
4,2018-03-29,407822,1,1,109,Arizona Diamondbacks,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,4.0,1.0,3.0,0.0,1.0,0.0,0.333333,42.0,0,0.0,0.0,0.0


(182950, 49)

In [29]:
# Group Result Tester... (just to validate the data)
groupby_player_game_stats = player_game_stats.groupby(['dailyDataDate','playerId'], as_index=False).agg(numGames=('gamePk','nunique'),
                                    numTeams = ('gameTeamId', 'nunique'),
                                    gameTeamId = ('gameTeamId', 'min'),
                                    gameTeamName = ('gameTeamName', 'min'))

display(player_game_stats.shape)
display(groupby_player_game_stats.shape)

# check the differcne 
x = player_date_stats_agg['numGames']
j = 0
for i in range (len(x)):
    if x[i] > 1:
        #print(i, x[i])
        j += 1
print(j)

print(player_game_stats.shape[0] - groupby_player_game_stats.shape[0])

# check result
if ((player_game_stats.shape[0] - groupby_player_game_stats.shape[0]) == j):
    print('Correct')
else:
    print('check once again')

(185144, 89)

(182950, 6)

2194
2194
Correct


Notable in game events:

In [30]:
# merge games with events to get schedued length of gmaes
events_plus = pd.merge(events, games[['gamePk', 'scheduledInnings']].drop_duplicates(),on = ['gamePk'], how = 'left')

# get current score from batting & pitching team perspectives
events_plus['battingTeamScore'] = np.where(events_plus['halfInning'] == 'bottom', events_plus['homeScore'], events_plus['awayScore'])
events_plus['pitchingTeamScore'] = np.where(events_plus['halfInning'] == 'bottom',events_plus['awayScore'], events_plus['homeScore'])

# consider 100 mph event
events_plus['pitches100mph'] = np.where((events_plus['type'] == 'pitch') & (events_plus['startSpeed'] >= 100), 1, 0)

# consider 450 ft event
events_plus['HRDist450ft'] = np.where((events_plus['event'] == 'Home Run') & (events_plus['totalDistance'] >= 450), 1, 0)

# Use game context/score logic to add fields for notable in-game events
events_plus['gameTyingRBI'] = np.where((events_plus['isPaOver'] == 1) & (events_plus['rbi'] > 0) & (events_plus['battingTeamScore'] < events_plus['pitchingTeamScore']) & 
                                       ((events_plus['battingTeamScore'] + events_plus['rbi']) == events_plus['pitchingTeamScore']), 1, 0)
                            # Start w/ batting team behind in score and look at cases where adding RBI ties score 

events_plus['goAheadRBI'] = np.where((events_plus['isPaOver'] == 1) & (events_plus['rbi'] > 0) & 
                                     (events_plus['battingTeamScore'] <= events_plus['pitchingTeamScore']) &          # Start w/ batting team not ahead in score (can be tied)...
                                     ((events_plus['battingTeamScore'] + events_plus['rbi']) > events_plus['pitchingTeamScore']),1, 0)  # ... and look at cases where adding RBI puts batting team ahead

# Add field to count walk-off (game-winning, game-ending) RBI
events_plus['walkoffRBI'] = np.where((events_plus['inning'] >= events_plus['scheduledInnings']) & (events_plus['halfInning'] == 'bottom') &(events_plus['goAheadRBI'] == 1),1, 0)
  
added_events_fields = ['pitches100mph', 'HRDist450ft', 'gameTyingRBI','goAheadRBI', 'walkoffRBI']

# Count overall frequency of added events
event_counts = events_plus[added_events_fields].sum()

display(event_counts)

pitches100mph     3265
HRDist450ft        365
gameTyingRBI      4030
goAheadRBI       11524
walkoffRBI         487
dtype: int64

#### Aggregate Player event-based stats to date level

In [31]:
# first, start with all pitchers (transform event based to date based stats)
pitcher_date_events_agg = (events_plus.groupby(['dailyDataDate', 'pitcherId'], as_index = False)
                           .agg(pitches100mph = ('pitches100mph', 'sum'),walkoffRBIAllowed = ('walkoffRBI', 'sum')  )  )

# secondly, start with all hitters (transform event based to date based stats)
hitter_date_events_agg = (events_plus.groupby(['dailyDataDate', 'hitterId'], as_index = False)[[field for field in added_events_fields if field != 'pitches100mph']].sum())
display(hitter_date_events_agg.head())

# merge both the type of players to one database (player dataset)
player_date_events_agg = (pd.merge(pitcher_date_events_agg.rename(columns = {'pitcherId': 'playerId'}),
                                   hitter_date_events_agg.rename(columns = {'hitterId': 'playerId'}),
                                   on = ['dailyDataDate', 'playerId'],how = 'outer').
                          fillna({field: 0 for field in added_events_fields + ['walkoffRBIAllowed']})) # NAs on events fields can be turned to 0 (no such stats in those categories)
  

display(player_date_events_agg.head())

,dailyDataDate,hitterId,HRDist450ft,gameTyingRBI,goAheadRBI,walkoffRBI
0,2018-03-29,134181,0,0,0,0
1,2018-03-29,400085,0,0,0,0
2,2018-03-29,400284,0,0,0,0
3,2018-03-29,405395,0,0,1,0
4,2018-03-29,408045,0,0,0,0


,dailyDataDate,playerId,pitches100mph,walkoffRBIAllowed,HRDist450ft,gameTyingRBI,goAheadRBI,walkoffRBI
0,2018-03-29,407822,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-03-29,407845,0.0,1.0,0.0,0.0,0.0,0.0
2,2018-03-29,430641,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-03-29,430935,0.0,0.0,0.0,0.0,0.0,0.0
4,2018-03-29,433587,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# Merge date-level player game stats with data-level player stats from events
player_date_stats_events_agg = (pd.merge(player_date_stats_agg,player_date_events_agg,
                                         on = ['dailyDataDate', 'playerId'],
                                         how = 'left'). fillna({field: 0 for field in added_events_fields + ['walkoffRBIAllowed']}))
                                                       # set event fields NAs to 0 (assumed since player has game stats but not added fields : added_events_fields = ['pitches100mph', 'HRDist450ft', 'gameTyingRBI','goAheadRBI', 'walkoffRBI'])
  
display(player_date_stats_events_agg.head())

,dailyDataDate,playerId,numGames,numTeams,gameTeamId,gameTeamName,gamesPlayedBatting,runsScored,doubles,triples,homeRuns,strikeOuts,baseOnBalls,hits,hitByPitch,atBats,caughtStealing,stolenBases,groundIntoDoublePlay,groundIntoTriplePlay,plateAppearances,totalBases,rbi,gamesPlayedPitching,gamesStartedPitching,completeGamesPitching,shutoutsPitching,winsPitching,lossesPitching,runsPitching,homeRunsPitching,strikeOutsPitching,baseOnBallsPitching,hitsPitching,earnedRuns,battersFaced,outsPitching,pitchesThrown,balls,strikes,saves,holds,blownSaves,inningsPitchedAsFrac,pitchingGameScore,noHitter,assists,putOuts,errors,pitches100mph,walkoffRBIAllowed,HRDist450ft,gameTyingRBI,goAheadRBI,walkoffRBI
0,2018-03-29,134181,1,1,140,Texas Rangers,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-03-29,400085,1,1,136,Seattle Mariners,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018-03-29,400284,1,1,119,Los Angeles Dodgers,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-03-29,405395,1,1,108,Los Angeles Angels,1.0,1.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2018-03-29,407822,1,1,109,Arizona Diamondbacks,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,4.0,1.0,3.0,0.0,1.0,0.0,0.333333,42.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# free some ram (delete sometable no logner needed)
del(events, events_plus, player_date_stats_agg, pitcher_date_events_agg,
  hitter_date_events_agg, player_date_events_agg)

gc.collect()

48

### Transactions Analytics
* Look at different transaction type values and frequency in data.

In [34]:
display(transactions.head(),transactions.shape)

,dailyDataDate,transactionId,playerId,playerName,date,fromTeamId,fromTeamName,toTeamId,toTeamName,effectiveDate,resolutionDate,typeCode,typeDesc,description
0,2018-01-01,340732,547348.0,C.C. Lee,2018-01-01,NaN,NaN,119,Los Angeles Dodgers,2018-01-01,2018-01-01,SFA,Signed as Free Agent,Los Angeles Dodgers signed free agent RHP C.C. Lee to a minor league contract and invited him to...
1,2018-01-02,339458,621173.0,Dylan Baker,2018-01-02,158.0,Milwaukee Brewers,119,Los Angeles Dodgers,2018-03-20,None,TR,Trade,Milwaukee Brewers traded RHP Dylan Baker to Los Angeles Dodgers for cash.
2,2018-01-02,357292,678876.0,Angel Rojas,2018-01-02,NaN,None,147,New York Yankees,2018-01-02,2018-01-02,SFA,Signed as Free Agent,New York Yankees signed free agent SS Angel Rojas to a minor league contract.
3,2018-01-02,341123,607054.0,Jace Peterson,2018-01-02,NaN,None,147,New York Yankees,2018-01-02,2018-01-02,SFA,Signed as Free Agent,New York Yankees signed free agent 2B Jace Peterson to a minor league contract and invited him t...
4,2018-01-02,339458,NaN,None,2018-01-02,119.0,Los Angeles Dodgers,158,Milwaukee Brewers,2018-01-02,None,TR,Trade,Milwaukee Brewers traded RHP Dylan Baker to Los Angeles Dodgers for cash.


(41030, 14)

In [37]:
# sort transaction types
transaction_type_values = transactions.groupby(['typeCode', 'typeDesc'], as_index = False)\
            .agg(numTransactions = ('date', 'count')).sort_values(['numTransactions'], ascending = False, ignore_index = True)

display(transaction_type_values)

,typeCode,typeDesc,numTransactions
0,ASG,Assigned,8598
1,SFA,Signed as Free Agent,7152
2,SC,Status Change,6898
3,OPT,Optioned,4527
4,CU,Recalled,4114
5,SGN,Signed,2090
6,SE,Selected,1716
7,TR,Trade,1622
8,DES,Designated for Assignment,1112
9,OUT,Outrighted,907


#### Create dataframe on player- date level with 1 column per transaction type...


In [38]:
# Pick certain transaction codes of interest from above list
transaction_types_of_interest = ['Assigned', 'Signed as Free Agent', 'Status Change', 'Optioned', 'Recalled', 'Signed', 'Selected', 'Trade', 'Designated for Assignment']

player_date_transactions_wide = (transactions.
  assign(
    # Create field w/ initial lower case & w/o spaces for later field names
    typeDescNoSpace = [(typeDesc[0].lower() + typeDesc[1:]) for typeDesc in transactions['typeDesc'].str.replace(' ', '')], count = 1)  # Add count ahead of pivot
  [
  # Filter to transactions of desired types and rows for actual players
    np.isin(transactions['typeDesc'], transaction_types_of_interest) & pd.notna(transactions['playerId'])] [['dailyDataDate', 'playerId', 'typeDescNoSpace', 'count']].
  # Filter to unique transaction types across player-date
  drop_duplicates().
 # Pivot data to 1 row per player-date and 1 column per transaction type
  pivot_table(
    index = ['dailyDataDate', 'playerId'],columns = 'typeDescNoSpace',values = 'count',fill_value = 0).reset_index())
                                                                               # NA can be turned to 0 since it means player didn't have that transaction that day
    
display(player_date_transactions_wide)

typeDescNoSpace,dailyDataDate,playerId,assigned,designatedforAssignment,optioned,recalled,selected,signed,signedasFreeAgent,statusChange,trade
0,2018-01-01,547348.0,0,0,0,0,0,0,1,0,0
1,2018-01-02,607054.0,0,0,0,0,0,0,1,0,0
2,2018-01-02,621173.0,0,0,0,0,0,0,0,0,1
3,2018-01-02,678876.0,0,0,0,0,0,0,1,0,0
4,2018-01-03,465753.0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
34181,2021-04-29,673357.0,0,0,0,0,0,0,0,1,0
34182,2021-04-29,676220.0,0,0,0,1,0,0,0,0,0
34183,2021-04-29,676424.0,0,0,0,1,0,0,0,1,0
34184,2021-04-29,676979.0,0,0,0,0,0,0,0,1,0


### Team standing

prepare team standing table for mergeing with player digital engagement data.
* will do the duplicacy check...


In [39]:
# Check for multiple entries for team standings on same date
team_dates_multiple_standings_entries = (standings.groupby(['dailyDataDate', 'teamId'], as_index = False)
                                         .agg(numTeamDateStandingsEntries = ('teamId', 'count')).query("numTeamDateStandingsEntries > 1"))
                                         
# If following returns 0 rows, can join to other daily data w/o worrying about duplicates 
display(team_dates_multiple_standings_entries)

# Pick only certain fields of interest from standings for merge
standings_selected_fields = (standings[['dailyDataDate', 'teamId', 'streakCode', 'divisionRank', 'leagueRank', 'wildCardRank', 'pct']].rename(columns = {'pct': 'winPct'}))
                                         
# Change column names to reflect these are all "team" standings - helps to differentiate from player-related fields if/when joining later
standings_selected_fields.columns = [(col_value + 'Team') if (col_value not in ['dailyDataDate', 'teamId']) else col_value for col_value in standings_selected_fields.columns.values]

standings_selected_fields['streakLengthTeam'] = (standings_selected_fields['streakCodeTeam'].str.replace('W', '').str.replace('L', '').astype(float))
                                         
# Add fields to separate winning and losing streak from streak code
standings_selected_fields['winStreakTeam'] = np.where(standings_selected_fields['streakCodeTeam'].str[0] == 'W',standings_selected_fields['streakLengthTeam'],np.nan)

standings_selected_fields['lossStreakTeam'] = np.where(standings_selected_fields['streakCodeTeam'].str[0] == 'L',standings_selected_fields['streakLengthTeam'],np.nan)


# Drop streak fields no longer necessary w/ derived values
standings_selected_fields.drop(['streakCodeTeam', 'streakLengthTeam'], axis = 1, inplace = True)

display(standings_selected_fields)                        

,dailyDataDate,teamId,numTeamDateStandingsEntries


,dailyDataDate,teamId,divisionRankTeam,leagueRankTeam,wildCardRankTeam,winPctTeam,winStreakTeam,lossStreakTeam
0,2018-03-29,112,1,3,3.0,1.000,1.0,NaN
1,2018-03-29,146,4,12,12.0,0.000,NaN,1.0
2,2018-03-29,121,2,5,5.0,1.000,1.0,NaN
3,2018-03-29,140,5,14,13.0,0.000,NaN,1.0
4,2018-03-29,144,1,2,2.0,1.000,1.0,NaN
...,...,...,...,...,...,...,...,...
15925,2021-04-30,138,2,6,4.0,0.538,2.0,NaN
15926,2021-04-30,109,4,5,3.0,0.538,2.0,NaN
15927,2021-04-30,137,1,2,NaN,0.615,NaN,1.0
15928,2021-04-30,120,3,11,8.0,0.455,2.0,NaN


### Awards

Look at various awards/honours received by player in daily data & berofe...


In [42]:
 player_awards_all = (pd.concat([awards[np.isin(awards['playerId'], players['playerId'])], 
                                award_pre2018.assign(dailyDataDate = pd.to_datetime(award_pre2018['awardDate'], format = '%Y-%m-%d'))], ignore_index = True).
                     sort_values(['awardDate'], ascending = False, ignore_index = True))
    
display(player_awards_all)
print('###########################################################################################')

awards_summary = player_awards_all.groupby(['awardId', 'awardName'], as_index = False)\
                  .agg(numWinnersInThisPlayerSet = ('playerId', 'count'),mostRecAwardDate = ('awardDate', 'max'),mostRecAwardSeason = ('awardSeason', 'max'))\
                  .sort_values(['mostRecAwardDate', 'awardId'],ascending = [False, True], ignore_index = True)

display(awards_summary)

,dailyDataDate,awardId,awardName,awardDate,awardSeason,playerId,playerName,awardPlayerTeamId
0,2021-04-30,ALROM,AL Rookie of the Month,2021-04-30,2021,606213,Yermin Mercedes,145.0
1,2021-04-30,NLPITOM,NL Pitcher of the Month,2021-04-30,2021,594798,Jacob deGrom,121.0
2,2021-04-30,NLRRELMON,NL Reliever of the Month,2021-04-30,2021,453343,Mark Melancon,135.0
3,2021-04-30,NLROM,NL Rookie of the Month,2021-04-30,2021,669432,Trevor Rogers,146.0
4,2021-04-30,ALPITOM,AL Pitcher of the Month,2021-04-30,2021,543037,Gerrit Cole,147.0
...,...,...,...,...,...,...,...,...
14046,2000-08-27,NLPOW,NL Player of the Week,2000-08-27,2000,134181,Adrian Beltre,119.0
14047,2000-07-09,FUTURES,Futures Game Selection,2000-07-09,2000,282332,CC Sabathia,402.0
14048,1998-09-01,TLPSAS,TEX Post-Season All-Star,1998-09-01,1998,134181,Adrian Beltre,510.0
14049,1998-07-07,ALAS,AL All-Star,1998-07-07,1998,112526,Bartolo Colon,114.0


###########################################################################################


,awardId,awardName,numWinnersInThisPlayerSet,mostRecAwardDate,mostRecAwardSeason
0,ALPITOM,AL Pitcher of the Month,60,2021-04-30,2021
1,ALPOM,AL Player of the Month,65,2021-04-30,2021
2,ALROM,AL Rookie of the Month,58,2021-04-30,2021
3,ALRRELMON,AL Reliever of the Month,21,2021-04-30,2021
4,NLPITOM,NL Pitcher of the Month,49,2021-04-30,2021
...,...,...,...,...,...
462,BAMAAABP,MiLB.com Triple-A Breakthrough Performer,1,2005-11-15,2005
463,BAMMIBP,MiLB.com Breakthrough Performer,1,2005-11-15,2005
464,PHIMVP,Phillies MVP Award,1,2005-10-18,2005
465,NYPPSAS,NYP Post-Season All-Star,3,2004-09-01,2004


Limit down the certain award categary.

In [43]:
uni_award_id = player_awards_all['awardId'].unique()
print(len(uni_award_id))

467


In [44]:
selected_awards = pd.DataFrame(data = {
    'awardId' : ['ALAS', 'NLAS', 'ALMVP', 'NLMVP', 'ALCY', 'NLCY'],
    'awardCategory' : ['AllStar', 'AllStar', 'MVP', 'MVP', 'CyYoung', 'CyYoung']
})

print('Main dataset of awards...')
display(player_awards_all.shape) # before execution
player_selected_awards = pd.merge(player_awards_all, selected_awards, on= 'awardId', how='inner')
print('for selected award id...')
display(player_selected_awards.shape) # after execution

selected_award_categories_in_data = (player_selected_awards['awardCategory'].unique())
print('unique data for selected award category')
display(selected_award_categories_in_data, selected_award_categories_in_data.shape)

player_selected_awards_by_date = (player_selected_awards.assign(count = 1).                              # Add count for use when pivoting
                                  pivot_table(index = ['dailyDataDate', 'playerId', 'playerName'],columns = 'awardCategory',values = 'count', fill_value = 0).reset_index())  # NA can be turned to 0 since it means player didn't get that award that day
print('pivot table...')
display(player_selected_awards_by_date)

# Add cumulative 'to date' sums for each award category
for award_category in selected_award_categories_in_data:
    player_selected_awards_by_date[('toDate' + award_category + 's')] = (player_selected_awards_by_date.groupby(['playerId', 'playerName'])[award_category].cumsum())
    
# Prepare for time-based merging by dropping non-"to date" fields
player_selected_awards_by_date.drop(selected_award_categories_in_data, axis = 1, inplace = True)


display(player_selected_awards_by_date)

Main dataset of awards...


(14051, 8)

for selected award id...


(745, 9)

unique data for selected award category


array(['MVP', 'CyYoung', 'AllStar'], dtype=object)

(3,)

pivot table...


awardCategory,dailyDataDate,playerId,playerName,AllStar,CyYoung,MVP
0,1998-07-07,112526,Bartolo Colon,1,0,0
1,2001-07-10,400085,Ichiro Suzuki,1,0,0
2,2001-07-10,405395,Albert Pujols,1,0,0
3,2001-11-01,400085,Ichiro Suzuki,0,0,1
4,2002-07-09,400085,Ichiro Suzuki,1,0,0
...,...,...,...,...,...,...
740,2019-11-14,641355,Cody Bellinger,0,0,1
741,2020-11-11,545333,Trevor Bauer,0,1,0
742,2020-11-11,669456,Shane Bieber,0,1,0
743,2020-11-12,518692,Freddie Freeman,0,0,1


awardCategory,dailyDataDate,playerId,playerName,toDateMVPs,toDateCyYoungs,toDateAllStars
0,1998-07-07,112526,Bartolo Colon,0,0,1
1,2001-07-10,400085,Ichiro Suzuki,0,0,1
2,2001-07-10,405395,Albert Pujols,0,0,1
3,2001-11-01,400085,Ichiro Suzuki,1,0,1
4,2002-07-09,400085,Ichiro Suzuki,1,0,2
...,...,...,...,...,...,...
740,2019-11-14,641355,Cody Bellinger,1,0,2
741,2020-11-11,545333,Trevor Bauer,0,1,1
742,2020-11-11,669456,Shane Bieber,0,1,1
743,2020-11-12,518692,Freddie Freeman,1,0,4


#### Player and Team twitter follower data merging with palyer daily engagement data....

In [45]:
# Extract only desired fields, rename some fields (for clarity later)
player_twitter_followers_for_merge = (playerTwitterFollowers[['dailyDataDate', 'date', 'playerId', 'numberOfFollowers']]
                                      .rename(columns = {'date': 'playerTwitterDataDate','numberOfFollowers': 'playerTwitterFollowers'}))  

# Extract only desired fields, rename some fields (for clarity/joining later)
team_twitter_followers_for_merge = (teamTwitterFollowers[['dailyDataDate', 'date', 'teamId', 'numberOfFollowers']].rename(columns = {'date': 'teamTwitterDataDate','teamId': 'rosterTeamIdIntForMerge','numberOfFollowers': 'teamTwitterFollowers'}))
                                                                                                                   # Name is weird, but helps set up for merge w/ digital engagement data

display(player_twitter_followers_for_merge.head())

display(team_twitter_followers_for_merge.head())

,dailyDataDate,playerTwitterDataDate,playerId,playerTwitterFollowers
0,2018-01-01,2018-01-01,545361,2452409
1,2018-01-01,2018-01-01,506433,1945081
2,2018-01-01,2018-01-01,434378,1795985
3,2018-01-01,2018-01-01,430897,1711807
4,2018-01-01,2018-01-01,120074,1515463


,dailyDataDate,teamTwitterDataDate,rosterTeamIdIntForMerge,teamTwitterFollowers
0,2018-01-01,2018-01-01,147,3130482
1,2018-01-01,2018-01-01,112,2373710
2,2018-01-01,2018-01-01,141,2196352
3,2018-01-01,2018-01-01,111,1950737
4,2018-01-01,2018-01-01,119,1949542


## Merge with other data with player daily engagement

In [46]:
# Merge in daily player engagement with date info, then filter to in-season dates only [PLAYER_ENGAGEMENT_WITH_INFO]
# Since test and future eval period is all 'in season', we imagine that looking at this filtered set of dates will help with identifying relevant trends more easily
player_engagement_with_info = (pd.merge(nextDayPlayerEngagement, dates_with_season_part,on = ['dailyDataDate'],how = 'left').query("inSeason").reset_index(drop = True))

# Take "row mean" across targets to add (helps with studying all 4 targets at once)
player_engagement_with_info['target1To4Avg'] = np.mean(player_engagement_with_info[['target1', 'target2', 'target3', 'target4']],axis = 1)

# Merge in some player information [PLAYERS]
player_engagement_with_info = pd.merge(player_engagement_with_info, players[['playerId', 'playerName', 'DOB', 'mlbDebutDate', 'birthCity','birthStateProvince', 'birthCountry', 'primaryPositionName']],
                                       on = ['playerId'], how = 'left')

# Merge in some player roster information by date [ROSTER]
player_engagement_with_info = pd.merge(player_engagement_with_info,(rosters[['dailyDataDate', 'playerId', 'statusCode', 'status', 'teamId']]
                                                                    .rename(columns = {'statusCode': 'rosterStatusCode','status': 'rosterStatus','teamId': 'rosterTeamId'})),on = ['dailyDataDate', 'playerId'],how = 'left')

# Add int version of rosterTeamId (w/ -1 for NA) to help w/ future merging []
player_engagement_with_info['rosterTeamIdIntForMerge'] = (np.where(pd.isna(player_engagement_with_info['rosterTeamId']), -1,player_engagement_with_info['rosterTeamId']).astype('int64'))

# Merge in team name from player's roster team [TEAMS]
player_engagement_with_info = pd.merge(player_engagement_with_info,(teams[['id', 'teamName']].rename(columns = {'id': 'rosterTeamId','teamName': 'rosterTeamName'})), on = ['rosterTeamId'],how = 'left')

# Merge in some player game stats and events (previously aggregated) from that date [PLAYER-DATE-EVENT-STAT]
player_engagement_with_info = pd.merge(player_engagement_with_info, player_date_stats_events_agg,on = ['dailyDataDate', 'playerId'],how = 'left')
    
# Merge in some team game stats/results (previously aggregated) from that date [TEAM-DATE-STAT]
player_engagement_with_info = pd.merge(player_engagement_with_info, team_date_stats_agg.rename(columns = {'teamId': 'gameTeamId'}), on = ['dailyDataDate', 'gameTeamId'], how = 'left')

# Get list of transactions fields to be added (and fill in NAs for post-merge) [* PLAYER-DATE-TRASACTION]
transactions_fields = (player_date_transactions_wide.drop(['dailyDataDate', 'playerId'] , axis = 1).columns.values.tolist())

# Merge in player transactions of note (previously created) on that date [ BOTH ]
player_engagement_with_info = (pd.merge(player_engagement_with_info,player_date_transactions_wide,on = ['dailyDataDate', 'playerId'],how = 'left')
                               .fillna({field: 0 for field in transactions_fields}))   # NAs on transactions fields can be turned to 0 (no player transaction that day)

# Merge in some pieces of team standings (previously created) from that date [SELECTED FIELDS]
player_engagement_with_info = pd.merge(player_engagement_with_info, standings_selected_fields.rename(columns = {'teamId': 'rosterTeamId'}), on = ['dailyDataDate', 'rosterTeamId'], how = 'left')
                                                                                                                # Join standings based on rosterTeamId (not gameTeamId) 

# Get list of awards fields to be added (and fill in NAs for post-merge)
awards_fields = (player_selected_awards_by_date.drop(['dailyDataDate', 'playerId', 'playerName'], axis = 1).columns.values.tolist())

# Merge in selected player awards received from latest award date before given date
player_engagement_with_info = (pd.merge_asof(player_engagement_with_info, player_selected_awards_by_date           # "merge" on date by player, looking backward (only use award dates up to daily date)
                                             .drop(['playerName'], axis = 1),on = ['dailyDataDate'], by = ['playerId'], direction = 'backward').fillna({field: 0 for field in awards_fields}))
                                                                                                                                  # NAs on awards fields can be turned to 0 (player had no awards of that type to date)

# Merge in player's Twitter followers from latest tracked date before given date [TWEETER-FOLOWER-DATA]
player_engagement_with_info = pd.merge_asof(player_engagement_with_info,player_twitter_followers_for_merge, on = ['dailyDataDate'], by = ['playerId'], direction = 'backward')
                                               # "merge" on date by player, looking backward (only use Twitter dates up to daily date)
    
#  Merge in team Twitter followers from latest date before given date 
player_engagement_with_info = pd.merge_asof(player_engagement_with_info, team_twitter_followers_for_merge, on = ['dailyDataDate'], by = ['rosterTeamIdIntForMerge'],direction = 'backward')
                                                         # "merge" on date by team, looking backward (only use Twitter dates up to daily date) 
                                                        # Use integer version of rosterTeamId since merge_asof seems to need int (not float)
    
# Drop integer version of rosterTeamId since merging is done
player_engagement_with_info.drop(['rosterTeamIdIntForMerge'], axis = 1)

display(player_engagement_with_info.head(), player_engagement_with_info.shape)

,dailyDataDate,engagementMetricsDate,playerId,target1,target2,target3,target4,year,season,month,inSeason,seasonPart,target1To4Avg,playerName,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,primaryPositionName,rosterStatusCode,rosterStatus,rosterTeamId,rosterTeamIdIntForMerge,rosterTeamName,numGames,numTeams,gameTeamId,gameTeamName,gamesPlayedBatting,runsScored,doubles,triples,homeRuns,strikeOuts,baseOnBalls,hits,hitByPitch,atBats,caughtStealing,stolenBases,groundIntoDoublePlay,groundIntoTriplePlay,plateAppearances,totalBases,rbi,gamesPlayedPitching,gamesStartedPitching,completeGamesPitching,shutoutsPitching,winsPitching,lossesPitching,runsPitching,homeRunsPitching,strikeOutsPitching,baseOnBallsPitching,hitsPitching,earnedRuns,battersFaced,outsPitching,pitchesThrown,balls,strikes,saves,holds,blownSaves,inningsPitchedAsFrac,pitchingGameScore,noHitter,assists,putOuts,errors,pitches100mph,walkoffRBIAllowed,HRDist450ft,gameTyingRBI,goAheadRBI,walkoffRBI,gameType,oppId,oppName,numGamesTeam,winsTeam,lossesTeam,runsScoredTeam,runsAllowedTeam,assigned,designatedforAssignment,optioned,recalled,selected,signed,signedasFreeAgent,statusChange,trade,divisionRankTeam,leagueRankTeam,wildCardRankTeam,winPctTeam,winStreakTeam,lossStreakTeam,toDateMVPs,toDateCyYoungs,toDateAllStars,playerTwitterDataDate,playerTwitterFollowers,teamTwitterDataDate,teamTwitterFollowers
0,2018-03-29,2018-03-30,608365,2.040838,7.893376,0.052359,2.599964,2018,2018,3,True,Reg Season 1st Half,3.146634,Addison Russell,1994-01-23,2015-04-21,Pensacola,FL,USA,Second Base,A,Active,112.0,112,Cubs,1.0,1.0,112.0,Chicago Cubs,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,3.0,0.0,1.0,0.0,0.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,5.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,R,146.0,Miami Marlins,1.0,1.0,0.0,8.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,1.0,1.0,NaN,0.0,0.0,1.0,2018-03-01,281424.0,2018-03-01,2437191.0
1,2018-03-29,2018-03-30,502624,1.068945,5.105543,2.722191,0.918550,2018,2018,3,True,Reg Season 1st Half,2.453807,Chase Anderson,1987-11-30,2014-05-11,Wichita Falls,TX,USA,Pitcher,A,Active,158.0,158,Brewers,1.0,1.0,158.0,Milwaukee Brewers,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,3.0,1.0,0.0,22.0,18.0,97.0,36.0,61.0,0.0,0.0,0.0,6.000000,74.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,R,135.0,San Diego Padres,1.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,4.0,1.0,1.0,NaN,0.0,0.0,0.0,2018-03-01,8118.0,2018-03-01,535654.0
2,2018-03-29,2018-03-30,643338,0.385945,5.368812,0.074404,0.565661,2018,2018,3,True,Reg Season 1st Half,1.598706,Chad Green,1991-05-24,2016-05-16,Greenville,SC,USA,Pitcher,A,Active,147.0,147,Yankees,1.0,1.0,147.0,New York Yankees,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,4.0,22.0,8.0,14.0,0.0,0.0,0.0,1.333333,51.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,R,141.0,Toronto Blue Jays,1.0,1.0,0.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,3.0,1.0,1.0,NaN,0.0,0.0,0.0,NaT,NaN,2018-03-01,3248455.0
3,2018-03-29,2018-03-30,458681,0.013176,2.200179,0.915358,0.539713,2018,2018,3,True,Reg Season 1st Half,0.917106,Lance Lynn,1987-05-12,2011-06-02,Marion County,IN,USA,Pitcher,A,Active,142.0,142,Twins,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,13.0,12.0,0.0,NaN,1.0,0.0,0.0,1.0,NaT,NaN,2018-03-01,592863.0
4,2018-03-29,2018-03-30,544925,0.000590,0.164543,0.000459,0.147902,2018,2018,3,True,Reg Season 1st Half,0.078374,Matthew den Dekker,1987-08-10,2013-08-29,Fort Lauderdale,FL,USA,Outfielder,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

(1168587, 108)

# Look at listing of all fields in merged player digital engagement data

In [47]:
display(player_engagement_with_info.info(max_cols = 50 ))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1168587 entries, 0 to 1168586
Columns: 108 entries, dailyDataDate to teamTwitterFollowers
dtypes: bool(1), datetime64[ns](3), float64(84), int64(5), object(15)
memory usage: 964.0+ MB


None

## Look at relationship between various Factors (features) and target variables

This relationship factors give an idea reagarding the variable/ feature importance, and help us to understand the best feature consideration for prediction model.

In [48]:
def GetGroupedTargetValuesSummary(df_with_group_and_targets, grouping_vars_list):
    # group target values by specified variables
    group_target_values_summary = (df_with_group_and_targets.groupby(grouping_vars_list, as_index = False, dropna = False).
                                   agg(numPlayerDates = ('playerId', 'count'), numPlayers = ('playerId', 'nunique'),
                                       target1 = ('target1', np.mean),
                                       target2 = ('target2', np.mean),
                                       target3 = ('target3', np.mean),
                                       target4 =  ('target4', np.mean),
                                       target1To4Avg = ('target1To4Avg', np.mean)).
                                   sort_values(['target1To4Avg'], ascending = False,ignore_index = True))
    return(group_target_values_summary)

def CalcAndDisplayMultipleSetsOfGroupedTargetValues(df_with_group_and_targets, grouping_names_and_vars_sets_df):
    num_group_sets = grouping_names_and_vars_sets_df.shape[0]
    
    kaggle_data_tabs = widgets.Tab()
    # Add Output widgets for each group as tabs' children
    kaggle_data_tabs.children = list([widgets.Output() for group_set in range(0, num_group_sets)])

    # Loop over each group (name/vars combo), create tab, calc/display summary
    for index, row in grouping_names_and_vars_sets_df.iterrows():
        this_group_name = row['groupName']
        this_group_vars_list = row['groupVarsList']
        # Group target values by specified variables
        group_target_values_summary = GetGroupedTargetValuesSummary(df_with_group_and_targets, this_group_vars_list) # 
        # Rename tab bar titles to string concatenating grouping vars
        kaggle_data_tabs.set_title(index, this_group_name)
        # Display corresponding table output for this tab name
        with kaggle_data_tabs.children[index]:
            print('Average Daily Digital Engagement by ' + this_group_name) 
            print('Average Target Values Grouped By ' + str(this_group_vars_list))
            display(group_target_values_summary.round(decimals = 1))
    
    display(kaggle_data_tabs)

### Get target averages for diffferent player-related factors (team/ toster/ transactions)


In [49]:
player_related_grouping_names_and_var_sets = pd.DataFrame(data = 
                                                          {'groupName': ['Player','Player Team (by Roster)','Player Roster Status','Player Selected Transactions This Day','Player Selected Awards to Date'],
                                                           'groupVarsList': pd.Series([['playerId', 'playerName'], ['rosterTeamName'],  ['rosterStatusCode', 'rosterStatus'],  ['trade', 'signedasFreeAgent', 'recalled'], ['toDateMVPs', 'toDateCyYoungs']])})    

CalcAndDisplayMultipleSetsOfGroupedTargetValues(player_engagement_with_info, player_related_grouping_names_and_var_sets)

### Conclusion 1: 

Some notes from the results above:

* Player ---> Well-known stars like Mike Trout, Aaron Judge, Bryce Harper, and Mookie Betts stand out with digital engagement averages of 17-25, much higher than the overall average across all players.

* Player_Team(by Rosters) ---> Players on some traditionally popular teams - Yankees, Dodgers, Red Sox - have had higher digital engagement averages, with some of the recently less successful teams with smaller fan bases - Pirates, Royals, Marlins - having lower digital engagement averages.

* Player Roster Status ---> Among roster status categories with significant sample size, 'active' players have the highest average digital engagement (as expected). Below them are players on the 10-day IL, players on the 60-day IL, and those reassigned to the minors (in that order).

* Player Selected Transaction ---> On average, traded players have much higher digital engagement right after the trade, compare to players who are signed as free agents (this is only in-season data, so free agency isn't as big), and both have higher averages than recalled players.

* Player Selected Award ---> Players with previous MVPs and Cy Young Awards tend to have much higher average digital engagement. It's hard to read into the value of having won those awards multiple times since there are only a few players who fit that category, but these "top-level" stars have much higher engagement than their peers in general.

### Get target averages for different player game info-related variables

In [50]:
player_game_related_grouping_names_and_var_sets = pd.DataFrame(data = {'groupName': ['Game Type','Player Team in Game','Player\'s Game Opponent','Player\'s Team Games, W-L',],
                                                                       'groupVarsList': pd.Series([['gameType'],['gameTeamName'],['oppName'],['numGamesTeam', 'winsTeam', 'lossesTeam']])
                                                                      })    

CalcAndDisplayMultipleSetsOfGroupedTargetValues(player_engagement_with_info, player_game_related_grouping_names_and_var_sets)

### Conclusion 2:

Some notes from the results above:

* Game_Type ---> Players in the All-Star Game (gameType = 'A') have had pretty high digital engagement on the day after that game. Outside of that, average player digital engagement (relative to peers) increases for each subsequent round of the postseason players are around for - the highest being the World Series (gameType = 'W') and the lowest being the regular season (gameType = 'R').

* Player Team in Game: ---> Outside of the All-Star teams, players playing in games (not just on the roster) for the Yankees, Dodgers, and Cubs have the highest digital engagement averages, while those playing for the Diamondbacks, Royals, and Marlins rate lowest on average (but above those players not playing in games). This table resembles that of the average digital engagement by players' roster team (above), but also has some key differences.

* Player's Game Opponent ---> The average digital engagement by game opponent list looks a good bit different, and has a bunch of teams clustered by division (e.g. all AL East teams up top, all AL Central teams at the bottom). This suggests that this could just be a byproduct of popularity of the teams that play those opponents more, since MLB teams play division opponents much more often than others.

* Player_Team Game Win/ Loss  ---> The effect isn't huge, but players whose teams won the day before tend to have higher digital engagement averages than those on losing teams.

## Get Target Averages for different player game batting-related stats

In [51]:
player_batting_related_grouping_names_and_var_sets = pd.DataFrame(data = 
                                                                  {'groupName': ['Player Games as Batter','Player Hits in Game','Player HR & Long HR in Game', 'Player Go-Ahead & Walkoff RBI in Game' ],
                                                                   'groupVarsList': pd.Series([['gamesPlayedBatting'],['hits'],['homeRuns', 'HRDist450ft'],['goAheadRBI', 'walkoffRBI']])})    

CalcAndDisplayMultipleSetsOfGroupedTargetValues(player_engagement_with_info, player_batting_related_grouping_names_and_var_sets)

## Conclusion 3:

* Player Game as a better ---> As expected, position players who actually play have higher digital engagement than those that don't play, with some additional bump for playing both games of a doubleheader (on average).

* Player hits in game ---> Average digital engagement is higher for players with each increasing number of hits on a date, up to 6 hits (keep in mind that doubleheaders mean this could be across 2 games).

* Player HR and long HR in game ---> Players who hit HR have higher average digital engagement, with multi-HR games even higher and the rare 3-HR game leading to very high averages. In a small sample, there seems to be some additional value to hitting long HR (defined here as those traveling 450+ ft).

* Payer Go-Ahead and Walkoff ---> Players have quite high digital engagement after recording a walkoff RBI, with some smaller (but still substantial) boost from having any type of go-ahead RBI in a game.

### Get target averages for different player game pitching-related stats

In [52]:
player_pitching_related_grouping_names_and_var_sets = pd.DataFrame(data = 
                                                                   {'groupName': ['Pitching Games Started & W-L', 'Pitcher Shutout in Game','Pitcher Complete Game & No-Hitter'],
                                                                    'groupVarsList': pd.Series([['gamesStartedPitching', 'winsPitching', 'lossesPitching'], ['shutoutsPitching'], ['completeGamesPitching', 'noHitter']])})    

CalcAndDisplayMultipleSetsOfGroupedTargetValues(player_engagement_with_info, player_pitching_related_grouping_names_and_var_sets)

## Conclusion 4:
* Pichting game start ---> Pitchers who starts, tend to have higher engagement than other players, with more than double the digital engagement (on average) if they record a win (vs a loss or no-decision).

* Pitcher Shutout in Game ---> The few pitchers who have thrown shutouts in this span have averaged very high digital engagement numbers the next day.

* Pitcher Comlete Game & No-Hitter ---> The 8 individual no-hitters in this span have resulted in player digital engagement numbers close to the max - an average of ~85 across all targets (on a 0-100 scale)! Other complete games (still pretty rare) having a pretty high average as well, near 15.

## Look at correlation coefficients of some continuous variables with target metrics

In [53]:
continuous_predictors = ['pitchesThrown', 'inningsPitchedAsFrac', 'strikeOutsPitching','pitchingGameScore', 'pitches100mph','plateAppearances', 
                         'hits', 'totalBases', 'winPctTeam', 'leagueRankTeam', 'winStreakTeam', 'lossStreakTeam','toDateAllStars', 
                         'playerTwitterFollowers', 'teamTwitterFollowers']

target_vars = ['target1', 'target2', 'target3', 'target4', 'target1To4Avg']

continuous_predictors_targets_correlations = (player_engagement_with_info[continuous_predictors + target_vars].corr().loc[continuous_predictors, target_vars])

display(continuous_predictors_targets_correlations.round(decimals = 2))

,target1,target2,target3,target4,target1To4Avg
pitchesThrown,-0.02,-0.05,0.08,-0.03,-0.00
inningsPitchedAsFrac,-0.01,-0.04,0.09,-0.03,0.01
strikeOutsPitching,0.01,-0.02,0.09,-0.01,0.03
pitchingGameScore,-0.09,-0.15,0.00,-0.11,-0.11
pitches100mph,0.01,0.02,0.02,0.03,0.02
plateAppearances,0.19,0.28,0.09,0.18,0.23
hits,0.24,0.26,0.13,0.15,0.25
totalBases,0.34,0.31,0.19,0.19,0.32
winPctTeam,0.08,0.09,0.08,0.09,0.11
leagueRankTeam,-0.10,-0.11,-0.09,-0.11,-0.13


## Intrpretation:
* Many of these stats/metrics, particularly the few pitching ones chosen here, have pretty low pairwise correlations with digital engagement. Keep in mind, this doesn't mean that these measurements have no predictive value. It could be that the relationship isn't linear or only exists at the extremes of a given metric (e.g. maybe having 10+ strikeouts as a pitcher means more toward digital engagement than any differences among single-digit numbers of strikeouts).

* There is a weak-to-moderate positive correlation between some basic hitting statistics (hits) that can take on a range of values on a day (plate appearances, hits, total bases) and digital engagement.

* There are some relatively weak correlations with team performance means digital engagement does not have high dependecne on team performance. Players on teams with higher win percentage or better league rank (lower is better) tend to have higher digital engagement, but the magnitude of the correlation is quite small.

* Players with more previous All-Star appearances and Twitter followers, and those on teams with higher followings on Twitter, tend to have higher average digital engagement. These are some of the stronger correlations among the factors considered here.

## **Next Step:**

- Now we will move to next step, which is generating ML model, to predict the target variables for digital engagement. (main goal of our competition).